<a href="https://colab.research.google.com/github/mjn6862/Freiburg_dataset/blob/master/Freiburg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

**Import Statements**

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import csv
import tensorflow_graphics.geometry.transformation as tfg_transformation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [10]:
#New from Kyle:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, custom_indices, batch_size=32,
                  shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/"
        self.indexes = custom_indices
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    def __getitem__(self, index):
        'Generate one batch of data'
        index = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        # Generate data
        X, y = self.__data_generation(index)#batch)
        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs)-5)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    def __data_generation(self, names_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        x1_full = np.empty((0, 480, 640, 4))
        x2_full = np.empty((0, 480, 640, 4))
        y_full = np.empty((0, 4, 4))
        for name in names_temp:
          x1 = np.load(self.direct+"/images/rgbd_%d.npy"%int(self.list_IDs[name][0]))
          x1 = np.reshape(x1,(1,480, 640, 4))
          x2 = np.load(self.direct+"images/rgbd_%d.npy"%int(self.list_IDs[name+5][0]))
          x2 = np.reshape(x2,(1,480, 640, 4))
          x1 = x1/255.
          x1 = x1 - np.mean(x1, axis=0)
          x2 = x2/255.
          x2 = x2 - np.mean(x2, axis=0)
          y1 = np.load(self.direct+"pose/htm_%d.npy"%int(self.list_IDs[name][0]))
          y2 = np.load(self.direct+"pose/htm_%d.npy"%int(self.list_IDs[name+5][0]))
          y = np.linalg.inv(y1).dot(y2)
          x1_full = np.concatenate((x1_full, x1))
          x2_full = np.concatenate((x2_full, x2))
          y_full = np.concatenate((y_full, np.reshape(y, (1, 4, 4)) ))
        y_quat = tfg_transformation.quaternion.from_rotation_matrix(y_full[:,0:3, 0:3])
        total = tf.keras.layers.concatenate((y_quat, y_full[:,0:3,3]), dtype='float64')
        return [x1_full[:,:,:,0:3], x1_full[:,:,:,3], x2_full[:,:,:,0:3], x2_full[:,:,:,3]], total

**Define the test-train split and create the Data Generator**

In [11]:
#New from Kyle
with open("./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/indices.csv", newline='') as f:
  reader = csv.reader(f)
  list_IDs = list(reader)
indexes = np.arange(len(list_IDs)-5)
np.random.shuffle(indexes)
# Define train/test split
train_portion = 0.8
train_indices = indexes[0:int(np.floor(len(indexes)*train_portion))]
train_IDs = [list_IDs[i] for i in train_indices]
valid_indices = indexes[int(np.floor(len(indexes)*train_portion)):]
valid_IDs = [list_IDs[i] for i in valid_indices]
training_generator = DataGenerator(list_IDs = train_IDs, custom_indices = np.arange(len(train_IDs)), batch_size=16, shuffle=True )
validation_generator = DataGenerator(list_IDs = valid_IDs, custom_indices = np.arange(len(valid_IDs)), batch_size=16, shuffle=True )

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

In [ ]:
def mean_sq_err(y_true, y_pred):
  #y_pred = 15. + y_pred*15.
  return tf.keras.backend.mean(tf.keras.backend.square(y_true - y_pred)) #+ metric_var(y_true, y_pred)#1./(0.001+tf.keras.backend.var(y_pred))

In [ ]:
def mean_psuedo_huber(y_true, y_pred):
  scale = 10
  huber = scale*scale*(tf.keras.backend.sqrt(1+tf.keras.backend.square((y_true-y_pred)/scale)) - 1)
  return tf.keras.backend.mean(huber)

In [ ]:
def metric_var(y_true, y_pred):
  #return 1./(0.001+tf.keras.backend.var(y_pred))
  epsilon = 0.001
  return tf.keras.backend.maximum((epsilon + tf.keras.backend.var(y_true))/(epsilon+tf.keras.backend.var(y_pred)), 1)

In [ ]:
def mean_psuedo_huber_var(y_true, y_pred):
  scale_factor = 10.
  return scale_factor1*mean_psuedo_huber(y_true, y_pred) + metric_var(y_true, y_pred)

In [ ]:
def create_side():
  side_input = tf.keras.layers.Input(shape=(480, 640, 4), name="side_input")
  pre_split_conv_1 = tf.keras.layers.Conv2D(16, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(side_input)
  pre_split_conv_1 = tf.keras.layers.MaxPool2D((2,2))(pre_split_conv_1)
  side_output = tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(pre_split_conv_1)
  return tf.keras.models.Model(inputs = side_input, outputs = side_output)

**Define the input layers**

In [ ]:
input_A = tf.keras.layers.Input(shape=(480, 640, 3), name="first_image")
depth_A = tf.keras.layers.Input(shape=(480, 640, 1), name="first_depth")
input_B = tf.keras.layers.Input(shape=(480, 640, 3), name="second_image")
depth_B = tf.keras.layers.Input(shape=(480, 640, 1), name="second_depth")

**Define the model**

In [ ]:
"""
The input to the pose estimation network is the target view concatenated with all the source views 
(along the color channels), and the outputs are the relative poses between the target view and each 
of the source views.
"""

concat = tf.keras.layers.concatenate([input_A, depth_A, input_B, depth_B])
"""
The network consists of 7 stride-2 convolutions 
"""
conv_1 = tf.keras.layers.Conv2D(16, (7,7), strides=(2,2), activation='relu')(concat)
conv_2 = tf.keras.layers.Conv2D(32, (5,5), strides=(2,2), activation='relu')(conv_1)
conv_3 = tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), activation='relu')(conv_2)
conv_4 = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), activation='relu')(conv_3)
conv_5 = tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), activation='relu')(conv_4)
conv_6 = tf.keras.layers.Conv2D(512, (3,3), strides=(2,2), activation='relu')(conv_5)
conv_7 = tf.keras.layers.Conv2D(1024, (3,3), strides=(2,2), activation='relu')(conv_6)
"""
followed by a 1×1 convolution with 6∗(N−1) output channels (corresponding to 3 Euler angles and 
3-D translation for each source view). 
"""
conv_8 = tf.keras.layers.Conv2D(7, (1,1), strides= (1,1))(conv_7)

"""
Finally, global average pooling is applied to aggregate predictions at all spatial locations. 
"""
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_8)

model = tf.keras.models.Model(inputs=[input_A, depth_A,  input_B, depth_B], outputs=avg_pool)


In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_image (InputLayer)        [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
first_depth (InputLayer)        [(None, 480, 640, 1) 0                                            
__________________________________________________________________________________________________
second_image (InputLayer)       [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
second_depth (InputLayer)       [(None, 480, 640, 1) 0                                            
____________________________________________________________________________________________

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [ ]:
_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=_optimizer, loss = mean_psuedo_huber,  metrics=[metric_var, mean_sq_err])

**Train using the fit_generator**

In [ ]:
%load_ext tensorboard

In [ ]:
logdir = "./drive/My Drive/SpeedChallenge/Logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, update_freq=1250)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./drive/My Drive/SpeedChallenge/CheckFreiburg/removed_activations_changed_pool{epoch}.h5", save_weights_only=True,  verbose=0)

In [ ]:
#New from Kyle:
model.fit(training_generator, epochs=4, verbose=1, initial_epoch=0, validation_data = validation_generator)

Epoch 1/4
59/59 [==============================] - 103s 2s/step - loss: 0.0585 - metric_var: 35.4881 - mean_sq_err: 0.1171 - val_loss: 0.0573 - val_metric_var: 30.5160 - val_mean_sq_err: 0.1148
Epoch 2/4
59/59 [==============================] - 104s 2s/step - loss: 0.0563 - metric_var: 27.0333 - mean_sq_err: 0.1128 - val_loss: 0.0552 - val_metric_var: 23.6313 - val_mean_sq_err: 0.1106
Epoch 3/4
59/59 [==============================] - 104s 2s/step - loss: 0.0544 - metric_var: 21.3138 - mean_sq_err: 0.1089 - val_loss: 0.0534 - val_metric_var: 19.0340 - val_mean_sq_err: 0.1070
Epoch 4/4
59/59 [==============================] - 104s 2s/step - loss: 0.0526 - metric_var: 17.2160 - mean_sq_err: 0.1054 - val_loss: 0.0518 - val_metric_var: 15.5486 - val_mean_sq_err: 0.1037


In [ ]:
x1 = np.load("./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/images/rgbd_10760.npy")/255.
x2 = np.load("./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/images/rgbd_10765.npy")/255.
p1 = np.load("./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/pose/htm_10760.npy")
p2 = np.load("./drive/My Drive/Machine_Learning_Projects/freiburg_dataset/pose/htm_10765.npy")
delta_p = np.linalg.inv(p1).dot(p2)
q = tfg_transformation.quaternion.from_rotation_matrix(delta_p[0:3, 0:3])
print(q)
print(delta_p[0:3,3])
y = model.predict([x1[:,:,0:3].reshape(1,480,640,3), x1[:,:,3].reshape(1,480,640,1), x2[:,:,0:3].reshape(1,480,640,3), x2[:,:,3].reshape(1,480,640,1)])
print(y)